<a href="https://colab.research.google.com/github/DeniseKelley/demo2/blob/master/MLCivilHW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Process area, day and month

In [111]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/forestfires.csv")

df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [112]:
# Assuming df is your DataFrame
if df.isna().any().any():
    print("There are missing values (NA) in the DataFrame.")
else:
    print("No missing values (NA) found in the DataFrame.")

No missing values (NA) found in the DataFrame.


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    517 non-null    float64
 5   DMC     517 non-null    float64
 6   DC      517 non-null    float64
 7   ISI     517 non-null    float64
 8   temp    517 non-null    float64
 9   RH      517 non-null    int64  
 10  wind    517 non-null    float64
 11  rain    517 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [114]:
#To reduce skewness and improve symmetry, the logarithm function that tends to improve
#regression results for right-skewed targets was applied to the area attribute
#logarithm function y = ln(x + 1)
df['area'] = np.log(df['area'] + 1)

In [115]:
# Create dummy variable for weekday/weekend based on the 'day' variable

df['weekday/weekend'] = df['day'].apply(lambda x: 1 if x in ['sat', 'sun'] else 0)
df = df.drop(['day'], axis = 1)




In [116]:
# Map months to seasons
season_mapping = {
    'jan': 'Winter', 'feb': 'Winter', 'mar': 'Spring',
    'apr': 'Spring', 'may': 'Spring', 'jun': 'Summer',
    'jul': 'Summer', 'aug': 'Summer', 'sep': 'Fall',
    'oct': 'Fall', 'nov': 'Fall', 'dec': 'Winter'
}
df['season'] = df['month'].map(season_mapping)

# Apply one-hot encoding to the 'season' column
df = pd.get_dummies(df, columns=['season'], prefix='season')

# Drop the original 'month' column
df = df.drop(['month'], axis=1)

# Display the first few rows of the transformed DataFrame
df.head()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,weekday/weekend,season_Fall,season_Spring,season_Summer,season_Winter
0,7,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,0,0,1,0,0
1,7,4,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,1,0,0,0
2,7,4,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,1,1,0,0,0
3,8,6,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,0,0,1,0,0
4,8,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,1,0,1,0,0


##2. Multicollinearity
[VIF less than 5 suggests that
multicollinearity should be of little concern].

In [101]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Selecting only numerical variables
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Adding a constant term for the intercept
X = add_constant(df[numerical_columns])

# Calculating VIF for each variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Displaying VIF results
print(vif_data)

           Variable         VIF
0             const  509.114393
1                 X    1.438229
2                 Y    1.445850
3              FFMC    1.705973
4               DMC    2.364794
5                DC    2.144246
6               ISI    1.586703
7              temp    2.729174
8                RH    1.959745
9              wind    1.152674
10             rain    1.052578
11             area    1.027980
12  weekday/weekend    1.045702


##3. Fit linear regression models
Please apply the forest fire data to fit linear regression models, conduct model selection by using three different methods (e.g., best subset selection, forward selection, and backward
selection), and report the best model for each method, denoted by BM1, BM2, and BM3

In [ ]:
%matplotlib inline
import statsmodels.api as sm
import itertools
from itertools import combinations
import time
import matplotlib.pyplot as plt

# Define the response variable
y = df['area']

# Define the predictor variables
X = df.drop(columns=['area'])


In [ ]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y, X[list(feature_set)])
    regr = model.fit()
    #RSS = ((regr.predict(X[list(feature_set)]) - y) ** 2).sum()
    #return {"model": regr, "RSS": RSS}

    # Make predictions
    predictions = regr.predict(X[list(feature_set)])

    # Calculate residuals
    residuals = y - predictions

    # Calculate RSS
    RSS = np.sum(residuals ** 2)

    # Calculate MAE
    MAE = np.mean(np.abs(residuals))

    # Calculate MSE
    MSE = np.mean(residuals ** 2)

    return {"model": regr, "RSS": RSS, "MAE": MAE, "MSE": MSE}

In [ ]:
def getBest(k):
    tic = time.time()
    results = []

    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))

    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)

    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]

    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    # Print MAE and MSE for the best model
    print("MAE:", best_model['MAE'])
    print("MSE:", best_model['MSE'])

    # Return the best model, along with some other useful information about the model
    return best_model

In [ ]:
# Could take quite awhile to complete...

BM1_best = pd.DataFrame(columns=["RSS", "model", "MAE", "MSE"])

tic = time.time()
for i in range(1,16):
    BM1_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")
mean_MAE_BM1 = BM1_best['MAE'].mean()
std_MAE_BM1 = BM1_best['MAE'].std()
mean_MSE_BM1 = BM1_best['MSE'].mean()
std_MSE_BM1 = BM1_best['MSE'].std()

print("Mean MAE for BM1:", mean_MAE_BM1)
print("Std MAE for BM1:", std_MAE_BM1)
print("Mean MSE for BM1:", mean_MSE_BM1)
print("Std MSE for BM1:", std_MSE_BM1)

Processed 15 models on 1 predictors in 0.04490518569946289 seconds.
MAE: 1.1564478351428944
MSE: 1.9475706273465554
Processed 105 models on 2 predictors in 0.29544568061828613 seconds.
MAE: 1.1476183087753729
MSE: 1.9347701438981857
Processed 455 models on 3 predictors in 1.282174825668335 seconds.
MAE: 1.1411678555669684
MSE: 1.916364188983489
Processed 1365 models on 4 predictors in 3.9934775829315186 seconds.
MAE: 1.1313692819100165
MSE: 1.8934608457387834
Processed 3003 models on 5 predictors in 9.627198219299316 seconds.
MAE: 1.1267535164421254
MSE: 1.8786734754857826
Processed 5005 models on 6 predictors in 15.744595050811768 seconds.
MAE: 1.126251832434799
MSE: 1.8699869036653218
Processed 6435 models on 7 predictors in 21.6091411113739 seconds.
MAE: 1.1228188137819413
MSE: 1.8679200244619116
Processed 6435 models on 8 predictors in 20.818732976913452 seconds.
MAE: 1.1227528455096774
MSE: 1.8651847198000726
Processed 5005 models on 9 predictors in 16.11975336074829 seconds.
MAE:

In [ ]:
# Gets the second element from each row ('model') and pulls out its rsquared attribute
BM1_best.apply(lambda row: row[1].rsquared, axis=1)

1     0.388752
2     0.392769
3     0.398546
4     0.405734
5     0.410375
6     0.413102
7     0.413750
8     0.414609
9     0.415335
10    0.416057
11    0.047698
12    0.047845
13    0.047962
14    0.047994
15    0.047995
dtype: float64

In [ ]:
print(BM1_best.loc[6, "model"].summary())
print(BM1_best.loc[9, "model"].summary())
print(BM1_best.loc[10, "model"].summary())

                                 OLS Regression Results                                
Dep. Variable:                   area   R-squared (uncentered):                   0.413
Model:                            OLS   Adj. R-squared (uncentered):              0.406
Method:                 Least Squares   F-statistic:                              59.95
Date:                Wed, 20 Mar 2024   Prob (F-statistic):                    4.20e-56
Time:                        01:42:14   Log-Likelihood:                         -895.39
No. Observations:                 517   AIC:                                      1803.
Df Residuals:                     511   BIC:                                      1828.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

*10* predictors have the highest R^2 but based on MAE and MSE beyond 9 predictors the decrease in MAE and MSE becomes less significant.

Both models have similar R-squared values, but Model with lower features has a slightly higher F-statistic, indicating that it explains the variance in the dependent variable slightly better.

Therefore, Model with 9 features might be considered slightly better based on the F-statistic. However, the difference between the two models is not substantial.

In [ ]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]

    tic = time.time()

    results = []

    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))

    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)

    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]

    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    # Print MAE and MSE for the best model
    print("MAE:", best_model['MAE'])
    print("MSE:", best_model['MSE'])

    # Return the best model, along with some other useful information about the model
    return best_model

In [ ]:
BM2_best = pd.DataFrame(columns=["RSS", "model", "MAE", "MSE"])

tic = time.time()
predictors = []

for i in range(1,len(X.columns)+1):
    BM2_best.loc[i] = forward(predictors)
    predictors = BM2_best.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  15 models on 1 predictors in 0.043326616287231445 seconds.
MAE: 1.1564478351428944
MSE: 1.9475706273465554
Processed  14 models on 2 predictors in 0.040103912353515625 seconds.
MAE: 1.1476183087753729
MSE: 1.9347701438981857
Processed  13 models on 3 predictors in 0.04451751708984375 seconds.
MAE: 1.1391975845556586
MSE: 1.919420842736594
Processed  12 models on 4 predictors in 0.03469562530517578 seconds.
MAE: 1.1328788324958878
MSE: 1.9089968685621144
Processed  11 models on 5 predictors in 0.03191184997558594 seconds.
MAE: 1.129555817235323
MSE: 1.9004758467042926
Processed  10 models on 6 predictors in 0.03696441650390625 seconds.
MAE: 1.1270922533843528
MSE: 1.8899628910040782
Processed  9 models on 7 predictors in 0.03129243850708008 seconds.
MAE: 1.124057596331891
MSE: 1.8853599365029108
Processed  8 models on 8 predictors in 0.026013851165771484 seconds.
MAE: 1.1219777836891303
MSE: 1.8727905962840155
Processed  7 models on 9 predictors in 0.023390769958496094 second

In [ ]:
print(BM2_best.loc[6, "model"].summary())
print(BM2_best.loc[9, "model"].summary())
print(BM2_best.loc[10, "model"].summary())
print(BM2_best.loc[11, "model"].summary())



                                 OLS Regression Results                                
Dep. Variable:                   area   R-squared (uncentered):                   0.407
Model:                            OLS   Adj. R-squared (uncentered):              0.400
Method:                 Least Squares   F-statistic:                              58.41
Date:                Wed, 20 Mar 2024   Prob (F-statistic):                    6.16e-55
Time:                        01:37:00   Log-Likelihood:                         -898.14
No. Observations:                 517   AIC:                                      1808.
Df Residuals:                     511   BIC:                                      1834.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

Although the difference in MSE between models with 10 and 11 predictors is small, the model with 11 predictors has a slightly lower MSE, indicating better performance in terms of minimizing prediction errors on average.

OLS Model with 9 features is better as it has the highest uncentered R-squared value of 0.415.

In [ ]:
def backward(predictors):

    tic = time.time()

    results = []

    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))

    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)

    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]

    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (toc-tic), "seconds.")
    # Print MAE and MSE for the best model
    print("MAE:", best_model['MAE'])
    print("MSE:", best_model['MSE'])
    # Return the best model, along with some other useful information about the model
    return best_model

In [ ]:
BM3_best = pd.DataFrame(columns=["RSS", "model", "MAE", "MSE"], index = range(1,len(X.columns)))

tic = time.time()
predictors = X.columns

while(len(predictors) > 1):
    BM3_best.loc[len(predictors)-1] = backward(predictors)
    predictors = BM3_best.loc[len(predictors)-1]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  15 models on 14 predictors in 0.05575227737426758 seconds.
MAE: 1.1171339121751351
MSE: 1.8581637783366214
Processed  14 models on 13 predictors in 0.05087995529174805 seconds.
MAE: 1.1173440038732672
MSE: 1.8582265270265663
Processed  13 models on 12 predictors in 0.04665350914001465 seconds.
MAE: 1.1176679614773806
MSE: 1.858455172760275
Processed  12 models on 11 predictors in 0.03955864906311035 seconds.
MAE: 1.117802274057963
MSE: 1.858741417866943
Processed  11 models on 10 predictors in 0.04158592224121094 seconds.
MAE: 1.1200897455443246
MSE: 1.8605685390205762
Processed  10 models on 9 predictors in 0.04213213920593262 seconds.
MAE: 1.1206315361289614
MSE: 1.8628721202627865
Processed  9 models on 8 predictors in 0.039865732192993164 seconds.
MAE: 1.1206700372539908
MSE: 1.8656463163355088
Processed  8 models on 7 predictors in 0.02899026870727539 seconds.
MAE: 1.123501275742953
MSE: 1.8760755206484578
Processed  7 models on 6 predictors in 0.026661396026611328 seco

In [ ]:
print(BM3_best.loc[11, "model"].summary())
print(BM3_best.loc[12, "model"].summary())
print(BM3_best.loc[9, "model"].summary())
print(BM3_best.loc[6, "model"].summary())

                            OLS Regression Results                            
Dep. Variable:                   area   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     2.534
Date:                Wed, 20 Mar 2024   Prob (F-statistic):            0.00549
Time:                        01:31:53   Log-Likelihood:                -893.84
No. Observations:                 517   AIC:                             1810.
Df Residuals:                     506   BIC:                             1856.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
X                   0.0453      0.026     

As we can observe, the MAE and MSE start to increase after the model with 12 predictors. Therefore, the model with 12 predictors is better among these options, as it achieves the lowest MAE and MSE before the metrics start to degrade.

Comparing the models, both have similar R-squared values, but Model with lower features has a slightly higher F-statistic. Additionally, it has more statistically significant coefficients compared to Model with more features.

I would select OLS Model 4 with 6 features as it has the highest uncentered R2 value (0.407) and the lowest AIC (1808) and BIC (1834) values among the four models. Additionally, it has a relatively high F-statistic (58.48), indicating that the model's overall fit is statistically significant.

## 4. BM1 vs BM2 vs BM3 vs STFWI, STM, FWI, and M

Please compare the best model(s) identified from Q3 along with the four linear regression
models (feature selection setups: STFWI, STM, FWI, and M, as specified in the paper) via 10-fold
cross-validation and report your results in a table below. [Please set random seed as 100 for
reproducibility]

In [ ]:
mean_MAE_BM1 = BM1_best['MAE'].mean()
std_MAE_BM1 = BM1_best['MAE'].std()
mean_MSE_BM1 = BM1_best['MSE'].mean()
std_MSE_BM1 = BM1_best['MSE'].std()

print("Mean MAE for BM1:", mean_MAE_BM1)
print("Std MAE for BM1:", std_MAE_BM1)
print("Mean MSE for BM1:", mean_MSE_BM1)
print("Std MSE for BM1:", std_MSE_BM1)

mean_MAE_BM2 = BM2_best['MAE'].mean()
std_MAE_BM2 = BM2_best['MAE'].std()
mean_MSE_BM2 = BM2_best['MSE'].mean()
std_MSE_BM2 = BM2_best['MSE'].std()

# Print results for BM2_best
print("Mean MAE for BM2:", mean_MAE_BM2)
print("Std MAE for BM2:", std_MAE_BM2)
print("Mean MSE for BM2:", mean_MSE_BM2)
print("Std MSE for BM2:", std_MSE_BM2)

# Calculate mean and std of MAE and MSE for BM3_best
mean_MAE_BM3 = BM3_best['MAE'].mean()
std_MAE_BM3 = BM3_best['MAE'].std()
mean_MSE_BM3 = BM3_best['MSE'].mean()
std_MSE_BM3 = BM3_best['MSE'].std()

# Print results for BM3_best
print("Mean MAE for BM3:", mean_MAE_BM3)
print("Std MAE for BM3:", std_MAE_BM3)
print("Mean MSE for BM3:", mean_MSE_BM3)
print("Std MSE for BM3:", std_MSE_BM3)

Mean MAE for BM1: 1.1268609133627778
Std MAE for BM1: 0.012239444210502938
Mean MSE for BM1: 1.879274717463875
Std MSE for BM1: 0.02993266276301667
Mean MAE for BM2: 1.128068232436584
Std MAE for BM2: 0.011526161798799629
Mean MSE for BM2: 1.8863055297033218
Std MSE for BM2: 0.029625360403333766
Mean MAE for BM3: 1.1294634563366457
Std MAE for BM3: 0.01393336642124385
Mean MSE for BM3: 1.886370601905274
Std MSE for BM3: 0.03235730919637446


    BM1:
        R-squared (uncentered): 0.415
        Adj. R-squared (uncentered): 0.405
        F-statistic: 40.11
        Prob (F-statistic): 6.60e-54
        AIC: 1807
        BIC: 1845

    BM2:
        R-squared (uncentered): 0.414
        Adj. R-squared (uncentered): 0.403
        F-statistic: 39.82
        Prob (F-statistic): 1.39e-53
        AIC: 1808
        BIC: 1847

    BM3:
        R-squared (uncentered): 0.415
        Adj. R-squared (uncentered): 0.405
        F-statistic: 40.11
        Prob (F-statistic): 6.60e-54
        AIC: 1807
        BIC: 1845

Based on the 9 predictors, Model 1 and Model 3 appear to be identical, while Model 2 performs slightly worse. Therefore, between Model 1 and Model 3, we can choose either of them. However, it's essential to consider the context of your analysis and the specific requirements of your problem when determining the best model.

**it seems that the model with 9 predictors performs slightly better than the model with 6 predictors.**

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold

# Set random seed for reproducibility
np.random.seed(100)

# Assuming df is your DataFrame containing the data
# Extract the features X and the target variable y
X = df.drop(columns=['area'])  # Assuming 'area' is the target variable
y = df['area']

# Get the feature names
feature_names = X.columns
threshold = 0.01

# Initialize 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize four linear regression models
models = {
    'STFWI': LinearRegression(),
    'STM': LinearRegression(),
    'FWI': LinearRegression(),
    'M': LinearRegression()
}

# Define feature selection methods for each model
feature_selection_methods = {
    'STFWI': SelectFromModel(estimator=LinearRegression()),
    'STM': SelectKBest(score_func=f_regression, k='all'),  # Use 'all' to select all features
    'FWI': SelectFromModel(estimator=LinearRegression(), threshold=threshold),
    'M': None  # No feature selection for model M
}

# Perform cross-validation and evaluate each model
for model_name, model in models.items():
    feature_selection_method = feature_selection_methods[model_name]

    if feature_selection_method is not None:
        X_selected = feature_selection_method.fit_transform(X, y)
        # Get the selected feature indices
        selected_indices = feature_selection_method.get_support(indices=True)
        selected_features = feature_names[selected_indices]
        print(f"Selected features for {model_name}: {selected_features}")
    else:
        X_selected = X

    # Compute MSE
    mse_scores = -cross_val_score(model, X_selected, y, cv=kf, scoring='neg_mean_squared_error')
    mse_mean = mse_scores.mean()
    mse_std = mse_scores.std()

    # Compute MAE
    mae_scores = -cross_val_score(model, X_selected, y, cv=kf, scoring='neg_mean_absolute_error')
    mae_mean = mae_scores.mean()
    mae_std = mae_scores.std()

    # Print MSE and MAE
    print(f"{model_name} - Mean MSE: {mse_mean:.4f}, Std MSE: {mse_std:.4f}, Mean MAE: {mae_mean:.4f}, Std MAE: {mae_std:.4f}")


Selected features for STFWI: Index(['weekday/weekend', 'season_Spring', 'season_Summer', 'season_Winter'], dtype='object')
STFWI - Mean MSE: 1.9647, Std MSE: 0.5080, Mean MAE: 1.1555, Std MAE: 0.1321
Selected features for STM: Index(['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain',
       'weekday/weekend', 'season_Fall', 'season_Spring', 'season_Summer',
       'season_Winter'],
      dtype='object')
STM - Mean MSE: 2.0758, Std MSE: 0.6657, Mean MAE: 1.1589, Std MAE: 0.1285
Selected features for FWI: Index(['X', 'FFMC', 'ISI', 'temp', 'wind', 'rain', 'weekday/weekend',
       'season_Fall', 'season_Spring', 'season_Summer', 'season_Winter'],
      dtype='object')
FWI - Mean MSE: 2.0415, Std MSE: 0.5971, Mean MAE: 1.1558, Std MAE: 0.1208
M - Mean MSE: 2.0758, Std MSE: 0.6657, Mean MAE: 1.1589, Std MAE: 0.1285


##5. Given the four feature selection setups, please develop the following GAM models and conduct 10-fold cross-validation and report your results in the following table. [Please set random seed as 100 for reproducibility] (20')


    •	STFWI_GAM: apply natural spline transformation to the “ISI” variable with 4 degrees of freedom, keep the others as linear terms.



In [ ]:
from patsy import dmatrix
import statsmodels.api as sm
import pandas as pd

# Assuming df is your DataFrame containing the data
# Extract the features X and the target variable y
X = df.drop(columns=['area'])  # Assuming 'area' is the target variable
y = df['area']

# Define linear terms to keep
linear_terms = ['weekday/weekend', 'season_Spring', 'season_Summer', 'season_Winter']

# Apply natural spline transformation to ISI with 4 degrees of freedom
transformed_ISI = dmatrix("bs(df.ISI, df=4, degree=3, include_intercept=False)",
                          {"df.ISI": df.ISI}, return_type='dataframe')

# Concatenate the transformed ISI feature with the specified linear terms
transformed_X = pd.concat([transformed_ISI, X[linear_terms]], axis=1)

# Initialize the GAM model
model = sm.GLM(y, transformed_X)

# Fit the model
fit = model.fit()

# Print the model parameters
print(fit.params)


Intercept                                                 0.432478
bs(df.ISI, df=4, degree=3, include_intercept=False)[0]    0.920544
bs(df.ISI, df=4, degree=3, include_intercept=False)[1]    0.147615
bs(df.ISI, df=4, degree=3, include_intercept=False)[2]    2.901281
bs(df.ISI, df=4, degree=3, include_intercept=False)[3]   -0.360551
weekday/weekend                                           0.130022
season_Spring                                            -0.367615
season_Summer                                            -0.219366
season_Winter                                             0.365608
dtype: float64


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from patsy import dmatrix
import statsmodels.api as sm

# Set random seed for reproducibility
np.random.seed(100)

X = df.drop(columns=['area'])  # Assuming 'area' is the target variable
y = df['area']

# Apply natural spline transformation to ISI with 4 degrees of freedom
transformed_ISI = dmatrix("bs(df.ISI, df=4, degree=3, include_intercept=False)", {"df.ISI": df.ISI}, return_type='dataframe')

# Keep other features as linear terms
linear_terms = ['weekday/weekend', 'season_Spring', 'season_Summer', 'season_Winter']

# Concatenate the transformed ISI feature with the specified linear terms
transformed_X = pd.concat([transformed_ISI, X[linear_terms]], axis=1)

# Initialize 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=100)

# Initialize lists to store MSE and MAE scores
mse_scores = []
mae_scores = []

# Perform cross-validation and evaluate the model
for train_index, test_index in kf.split(transformed_X):
    X_train, X_test = transformed_X.iloc[train_index], transformed_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize the GAM model
    model = sm.GLM(y_train, X_train)

    # Fit the model
    fit = model.fit()

    # Make predictions
    y_pred = fit.predict(X_test)

    # Compute MSE and MAE
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Append scores to the lists
    mse_scores.append(mse)
    mae_scores.append(mae)

# Calculate mean and standard deviation of MSE and MAE
mse_mean = np.mean(mse_scores)
mse_std = np.std(mse_scores)
mae_mean = np.mean(mae_scores)
mae_std = np.std(mae_scores)

# Print the results
print("STFWI_GAM Results:")
print(f"Mean MSE: {mse_mean:.4f}, Std MSE: {mse_std:.4f}")
print(f"Mean MAE: {mae_mean:.4f}, Std MAE: {mae_std:.4f}")



STFWI_GAM Results:
Mean MSE: 4.4449, Std MSE: 7.2501
Mean MAE: 1.2260, Std MAE: 0.1633


    •	STM_GAM: apply natural spline transformation to the “temp” variable with 4 degrees of freedom, apply smooth spline transformation to the “wind” variable with 5 degrees of freedom, keep the others as linear terms.


In [ ]:
# Set random seed for reproducibility
np.random.seed(100)

X = df.drop(columns=['area'])  # Assuming 'area' is the target variable
y = df['area']

# Define linear terms to keep
linear_terms = ['X', 'FFMC', 'ISI', 'temp', 'wind', 'rain', 'weekday/weekend',
                'season_Fall', 'season_Spring', 'season_Summer', 'season_Winter']

# Apply natural spline transformation to temp with 4 degrees of freedom
transformed_temp = dmatrix("bs(df.temp, df=4, degree=3, include_intercept=False)",
                           {"df.temp": df.temp}, return_type='dataframe')

# Apply smooth spline transformation to wind with 5 degrees of freedom
transformed_wind = dmatrix("cr(df.wind, df=5)", {"df.wind": df.wind}, return_type='dataframe')

# Concatenate the transformed temp and wind features with the specified linear terms
transformed_X = pd.concat([transformed_temp, transformed_wind, X[linear_terms]], axis=1)

# Initialize 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=100)

# Initialize lists to store MSE and MAE scores
mse_scores = []
mae_scores = []

# Perform cross-validation and evaluate the model
for train_index, test_index in kf.split(transformed_X):
    X_train, X_test = transformed_X.iloc[train_index], transformed_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize the GAM model
    model = sm.GLM(y_train, X_train)

    # Fit the model
    fit = model.fit()

    # Make predictions
    y_pred = fit.predict(X_test)

    # Compute MSE and MAE
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Append scores to the lists
    mse_scores.append(mse)
    mae_scores.append(mae)

# Calculate mean and standard deviation of MSE and MAE
mse_mean = np.mean(mse_scores)
mse_std = np.std(mse_scores)
mae_mean = np.mean(mae_scores)
mae_std = np.std(mae_scores)

# Print the results
print("STM_GAM Results:")
print(f"Mean MSE: {mse_mean:.4f}, Std MSE: {mse_std:.4f}")
print(f"Mean MAE: {mae_mean:.4f}, Std MAE: {mae_std:.4f}")

STM_GAM Results:
Mean MSE: 2.2006, Std MSE: 0.7809
Mean MAE: 1.1531, Std MAE: 0.1176


    •	FWI_GAM: apply natural spline transformation to the “ISI” variable with 4 degrees of freedom, keep the others as linear terms.

In [ ]:


# Set random seed for reproducibility
np.random.seed(100)

# Assuming df is your DataFrame containing the data
# Extract the features X and the target variable y
X = df.drop(columns=['area'])  # Features
y = df['area']  # Target variable

# Apply natural spline transformation to ISI with 4 degrees of freedom
transformed_ISI = dmatrix("bs(df.ISI, df=4, degree=3, include_intercept=False)",
                          {"df.ISI": df.ISI}, return_type='dataframe')

# Define linear terms to keep (all features selected)
linear_terms = X.columns.tolist()

# Concatenate the transformed ISI feature with the specified linear terms
transformed_X = pd.concat([transformed_ISI, X[linear_terms]], axis=1)

# Initialize 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=100)

# Initialize lists to store MSE and MAE scores
mse_scores = []
mae_scores = []

# Perform cross-validation and evaluate the model
for train_index, test_index in kf.split(transformed_X):
    X_train, X_test = transformed_X.iloc[train_index], transformed_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize the GAM model
    model = sm.GLM(y_train, X_train)

    # Fit the model
    fit = model.fit()

    # Make predictions
    y_pred = fit.predict(X_test)

    # Compute MSE and MAE
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Append scores to the lists
    mse_scores.append(mse)
    mae_scores.append(mae)

# Calculate mean and standard deviation of MSE and MAE
mse_mean = np.mean(mse_scores)
mse_std = np.std(mse_scores)
mae_mean = np.mean(mae_scores)
mae_std = np.std(mae_scores)

# Print the results
print("FWI_GAM Results:")
print(f"Mean MSE: {mse_mean:.4f}, Std MSE: {mse_std:.4f}")
print(f"Mean MAE: {mae_mean:.4f}, Std MAE: {mae_std:.4f}")


FWI_GAM Results:
Mean MSE: 2.3709, Std MSE: 0.8816
Mean MAE: 1.1873, Std MAE: 0.1014


    •	M_GAM: apply natural spline transformation to the “temp” variable with 4 degrees of freedom, apply smooth spline transformation to the “wind” variable with 5 degrees of freedom, keep the others as linear terms.


In [ ]:

# Set random seed for reproducibility
np.random.seed(100)

# Assuming df is your DataFrame containing the data
# Extract the features X and the target variable y
X = df.drop(columns=['area'])  # Features
y = df['area']  # Target variable

# Apply natural spline transformation to temp with 4 degrees of freedom
transformed_temp = dmatrix("bs(df.temp, df=4, degree=3, include_intercept=False)",
                           {"df.temp": df.temp}, return_type='dataframe')

# Apply smooth spline transformation to wind with 5 degrees of freedom
transformed_wind = dmatrix("cr(df.wind, df=5)", {"df.wind": df.wind}, return_type='dataframe')

# Define linear terms to keep (all features selected)
linear_terms = X.columns.tolist()

# Concatenate the transformed features with the specified linear terms
transformed_X = pd.concat([transformed_temp, transformed_wind, X[linear_terms]], axis=1)

# Initialize 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=100)

# Initialize lists to store MSE and MAE scores
mse_scores = []
mae_scores = []

# Perform cross-validation and evaluate the model
for train_index, test_index in kf.split(transformed_X):
    X_train, X_test = transformed_X.iloc[train_index], transformed_X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Initialize the GAM model
    model = sm.GLM(y_train, X_train)

    # Fit the model
    fit = model.fit()

    # Make predictions
    y_pred = fit.predict(X_test)

    # Compute MSE and MAE
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Append scores to the lists
    mse_scores.append(mse)
    mae_scores.append(mae)

# Calculate mean and standard deviation of MSE and MAE
mse_mean = np.mean(mse_scores)
mse_std = np.std(mse_scores)
mae_mean = np.mean(mae_scores)
mae_std = np.std(mae_scores)

# Print the results
print("M_GAM Results:")
print(f"Mean MSE: {mse_mean:.4f}, Std MSE: {mse_std:.4f}")
print(f"Mean MAE: {mae_mean:.4f}, Std MAE: {mae_std:.4f}")


M_GAM Results:
Mean MSE: 2.2340, Std MSE: 0.8340
Mean MAE: 1.1582, Std MAE: 0.1201


## 6. We now consider the regression trees. Given the four feature selection setups, please develop the following regression tree models (i.e., STFWI_GAM, STM_GAM, FWI_GAM, M_GAM) and conduct 10-fold cross-validation

In [124]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
import pandas as pd

from sklearn.metrics import mean_squared_error, mean_absolute_error



# Define feature sets for each model setup
feature_set_M_GAM = ['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain',
                     'weekday/weekend', 'season_Fall', 'season_Spring', 'season_Summer', 'season_Winter']

feature_set_FWI_GAM = ['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain',
                     'weekday/weekend', 'season_Fall', 'season_Spring', 'season_Summer', 'season_Winter']

linear_terms_STM_GAM = ['X', 'FFMC', 'ISI', 'temp', 'wind', 'rain', 'weekday/weekend',
                        'season_Fall', 'season_Spring', 'season_Summer', 'season_Winter']

linear_terms_STFWI_GAM = ['weekday/weekend', 'season_Spring', 'season_Summer', 'season_Winter']

# Split data into features (X) and target variable (y)
X = df.drop(columns=['area'])  # Features
y = df['area']  # Target variable

# Perform 10-fold cross-validation for each model setup
cv_scores = []

# Function to calculate MSE, MAE, Mean, and SD
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mean_mse = np.mean(mse)
    sd_mse = np.std(mse)
    mean_mae = np.mean(mae)
    sd_mae = np.std(mae)
    return mse, mae, mean_mse, sd_mse, mean_mae, sd_mae

# Perform 10-fold cross-validation for each model setup and calculate metrics
cv_metrics = []

def perform_cross_validation_with_metrics(X, y, model):
    mse_scores = []
    mae_scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse, mae, mean_mse, sd_mse, mean_mae, sd_mae = calculate_metrics(y_test, y_pred)
        mse_scores.append(mse)
        mae_scores.append(mae)
    return mse_scores, mae_scores

# Perform 10-fold cross-validation for each model setup and calculate metrics
model_M_GAM = DecisionTreeRegressor()
mse_M_GAM, mae_M_GAM = perform_cross_validation_with_metrics(X, y, model_M_GAM)

model_STM_GAM = DecisionTreeRegressor()
mse_STM_GAM, mae_STM_GAM = perform_cross_validation_with_metrics(df[linear_terms_STM_GAM], y, model_STM_GAM)

model_STFWI_GAM = DecisionTreeRegressor()
mse_STFWI_GAM, mae_STFWI_GAM = perform_cross_validation_with_metrics(df[linear_terms_STFWI_GAM], y, model_STFWI_GAM)

# Results
print("Mean Squared Error (MSE) and Mean Absolute Error (MAE) along with their Mean and Standard Deviation (SD):")
print("M_GAM: MSE - Mean = {:.4f}, SD = {:.4f}, MAE - Mean = {:.4f}, SD = {:.4f}".format(np.mean(mse_M_GAM), np.std(mse_M_GAM), np.mean(mae_M_GAM), np.std(mae_M_GAM)))
print("STM_GAM: MSE - Mean = {:.4f}, SD = {:.4f}, MAE - Mean = {:.4f}, SD = {:.4f}".format(np.mean(mse_STM_GAM), np.std(mse_STM_GAM), np.mean(mae_STM_GAM), np.std(mae_STM_GAM)))
print("STFWI_GAM: MSE - Mean = {:.4f}, SD = {:.4f}, MAE - Mean = {:.4f}, SD = {:.4f}".format(np.mean(mse_STFWI_GAM), np.std(mse_STFWI_GAM), np.mean(mae_STFWI_GAM), np.std(mae_STFWI_GAM)))

Mean Squared Error (MSE) and Mean Absolute Error (MAE) along with their Mean and Standard Deviation (SD):
M_GAM: MSE - Mean = 3.9369, SD = 0.9913, MAE - Mean = 1.3785, SD = 0.1387
STM_GAM: MSE - Mean = 4.0901, SD = 1.1077, MAE - Mean = 1.4238, SD = 0.1945
STFWI_GAM: MSE - Mean = 1.9808, SD = 0.4102, MAE - Mean = 1.1609, SD = 0.0824
